In [1]:
!pip install pandas numpy matplotlib seaborn scikit-learn xgboost shap

Active code page: 1252
   ---------------------------------------- 0.0/547.0 kB ? eta -:--:--
   ------------------- -------------------- 262.1/547.0 kB ? eta -:--:--
   ---------------------------------------- 547.0/547.0 kB 3.7 MB/s  0:00:00

   ---------------------------------------- 0/3 [slicer]
   ------------- -------------------------- 1/3 [cloudpickle]
   -------------------------- ------------- 2/3 [shap]
   -------------------------- ------------- 2/3 [shap]
   -------------------------- ------------- 2/3 [shap]
   -------------------------- ------------- 2/3 [shap]
   -------------------------- ------------- 2/3 [shap]
   -------------------------- ------------- 2/3 [shap]
   -------------------------- ------------- 2/3 [shap]
   -------------------------- ------------- 2/3 [shap]
   -------------------------- ------------- 2/3 [shap]
   -------------------------- ------------- 2/3 [shap]
   -------------------------- ------------- 2/3 [shap]
   -------------------------- -

In [17]:
DATA_PATH = "../data/raw/"

orders = pd.read_csv(DATA_PATH + "olist_orders_dataset.csv")
reviews = pd.read_csv(DATA_PATH + "olist_order_reviews_dataset.csv")

df = orders.merge(reviews, on="order_id", how="inner")

print("Total Orders with Reviews:", df.shape[0])

Total Orders with Reviews: 99224


In [18]:
df["low_rating"] = (df["review_score"] <= 2).astype(int)

distribution = df["low_rating"].value_counts(normalize=True)
distribution

low_rating
0    0.85311
1    0.14689
Name: proportion, dtype: float64

In [19]:
orders["order_purchase_timestamp"] = pd.to_datetime(orders["order_purchase_timestamp"])
orders["order_delivered_customer_date"] = pd.to_datetime(orders["order_delivered_customer_date"])
orders["order_estimated_delivery_date"] = pd.to_datetime(orders["order_estimated_delivery_date"])

orders["delivery_delay_days"] = (
    orders["order_delivered_customer_date"] -
    orders["order_estimated_delivery_date"]
).dt.days

df = df.merge(
    orders[["order_id", "delivery_delay_days"]],
    on="order_id",
    how="left"
)

df.groupby("low_rating")["delivery_delay_days"].mean()

low_rating
0   -12.912736
1    -5.149879
Name: delivery_delay_days, dtype: float64

Business Understanding Summary
In this notebook, I explored the marketplace dataset to identify a high-impact ML problem aligned with the customer's concerns.

Key findings:
Total reviewed orders: ~99K
Low ratings (≤2 stars): ~14.7% of orders
Delivery performance shows a meaningful difference between low and high ratings
Negative reviews directly impact seller satisfaction and marketplace margins
Based on these observations, I evaluated multiple potential ML problems.

Considered Problems
Revenue prediction
Customer churn prediction
Delivery delay prediction
Low review prediction

Final Decision
I selected low review score prediction (review_score ≤ 2) as the primary ML problem because:
It directly addresses bad reviews mentioned by the customer
It aligns with margin pressure and seller dissatisfaction
It provides measurable and actionable business impact
It is supported by clear signals in the data

Target definition:
low_rating = 1 if review_score <= 2 else 
This framing allows proactive intervention before negative feedback impacts platform trust.